In [2]:
import nltk
import keras
from nltk.corpus import stopwords
from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding
from keras.preprocessing import sequence
from keras.datasets import imdb
from sklearn.naive_bayes import GaussianNB
import numpy as np
import os
swords = set(stopwords.words('english'))

Using TensorFlow backend.


# Parsing and pre - processing

In [77]:
neg_list = os.listdir("neg_test/")
pos_list = os.listdir("pos_test/")

tot_text = []
for i in pos_list:
    fp = open("pos_test/"+i,"r")
    tot_text.append((fp.read(),1))
for i in neg_list:
    fp = open("neg_test/"+i,"r")
    tot_text.append((fp.read(),0))

print(tot_text[0])

clean_text = [(["".join([k.lower() for k in list(j) if k not in punctuation]) for j in i[0].split()],i[1]) for i in tot_text]
clean_text = [(" ".join(list(filter(lambda x:x not in swords,i[0]))),i[1]) for i in clean_text]
print()
print(clean_text[0])

text0map = [i[0] for i in clean_text]
text1map = [i[1] for i in clean_text]

("I was so moved by this film in 1981, I went back to the theater four times to see it again! Something I have never done for another film. No movie evokes the feelings of growing up in the 60's like Four Friends. That it so closely approximated my own experiences in the 60's is probably something that many will share. Jodi Thelen is radiantly beautiful and unforgetable! Why she didn't become a major star after this I will never know. The acting by the entire cast is flawless as is Steve Tisch's script. I always wanted to know how much of the story was autobiographical. But alas, Steve is no longer here to answer that question. I have all but worn out my VHS copy of this great movie! Highly recommended!", 1)

('moved film 1981 went back theater four times see something never done another film movie evokes feelings growing 60s like four friends closely approximated experiences 60s probably something many share jodi thelen radiantly beautiful unforgetable didnt become major star never kn

In [58]:
neg_list = os.listdir("neg_train/")
pos_list = os.listdir("pos_train/")

tot_text = []
for i in pos_list:
    fp = open("pos_train/"+i,"r")
    tot_text.append((fp.read(),1))
for i in neg_list:
    fp = open("neg_train/"+i,"r")
    tot_text.append((fp.read(),0))


clean_text = [(["".join([k.lower() for k in list(j) if k not in punctuation]) for j in i[0].split()],i[1]) for i in tot_text]
clean_text = [(" ".join(list(filter(lambda x:x not in swords,i[0]))),i[1]) for i in clean_text]


text0map0 = [i[0] for i in clean_text]
text1map1 = [i[1] for i in clean_text]

# Tokenizer to convert words to integers

In [75]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text0map + text0map0) 
sequence = tokenizer.texts_to_sequences(text0map + text0map0)
print(sequence[0])
get_word_index("something")

[1470, 3, 5214, 295, 55, 651, 570, 117, 11, 52, 35, 126, 63, 3, 2, 7023, 1240, 1698, 1583, 5, 570, 229, 3011, 61511, 2257, 1583, 136, 52, 31, 1359, 19827, 45461, 32474, 201, 51714, 60, 304, 519, 235, 35, 41, 37, 314, 86, 3566, 1191, 81606, 131, 106, 341, 41, 13, 12, 12880, 2953, 1191, 1009, 1321, 779, 4926, 1650, 825, 16, 2, 417, 1015]


In [3]:

top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)


In [4]:

max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)


# Deep learning with LSTM

In [6]:

embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 190s 8ms/step - loss: 0.4392 - acc: 0.7947 - val_loss: 0.3956 - val_acc: 0.8295
Epoch 2/3
25000/25000 [==============================] - 224s 9ms/step - loss: 0.3436 - 

In [61]:

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy:",(scores[1]*100))

Accuracy: 86.408


# Naive Bayes Classifier

In [15]:
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

## Convert text back to integers for translating

In [9]:
word_to_id = keras.datasets.imdb.get_word_index()

id_to_word = {v:k for k,v in word_to_id.items()}    
id_to_word[0] = "<pad>"
id_to_word[1] = "<empty>"
id_to_word[2] = "<na>"
  
full_text = {}



In [12]:
for i in range(25000):
    restored_text = " ".join([id_to_word[j] for j in X_test[i]])
    full_text[restored_text] = X_test[i]


In [13]:
def predictText(text):
    
    
    convert = [word_to_id[i] for i in text.split() if i in word_to_id]
    if len(convert)<500:
        convert += [0 for i in range(500-len(convert))]
    else:
        convert = convert[:500]
    
    sentiment = clf.predict([np.array(convert)])
    
    if sentiment:
        return "Positive"
    return "Negative"



In [17]:
predictText("this is so bad")

'Negative'

In [74]:
correct = 0
wrong = 0
val = 0
for i in range(len(text0map)):

    txt = predictText(text0map[i])
    if txt=="Positive":
        val=1
    else:
        val=0
    if val==text1map[i]:
        correct+=1
    else:
        wrong+=1
        
print("Accuracy : ", correct/(correct+wrong)*100)

        




Accuracy :  50.032
